In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.environ['ISX'] = '0'

import numpy as np
import pandas as pd
from scipy import stats

from dewan_calcium.helpers.project_folder import ProjectFolder
from dewan_calcium import classifiers
pd.options.mode.copy_on_write = "warn"

print('Finished importing required libraries!')

In [ ]:
# Create Project Folder to Gather and Hold all the File Paths
project_folder = ProjectFolder('ODOR', combined=True)

In [ ]:
# If this is the first time the project folder has been created,
# move the files to the appropriate directories and then run this cell, otherwise skip this cell
project_folder.get_data()

In [ ]:
cell_class = 'VGAT'
data_file = []
for file in project_folder.raw_data_dir.combined_data_path:
    if cell_class.lower() in str(file).lower():
        data_file = file
        break
        
if not data_file:
    raise FileExistsError(f'No data file with class {cell_class} exists!')

In [ ]:
combined_data = pd.read_pickle(data_file, compression={'method': 'xz'})
z_scored_combined_data = combined_data.T.groupby(level=0, group_keys=False).apply(stats.zscore).T
# Transform our dataframe to put the cells/odors as the index, group by level=0 (cell names), apply stats.zscore to each group, transform back

cells = np.unique(combined_data.columns.get_level_values(0).values)
odors = np.unique(combined_data.columns.get_level_values(1).values)


In [ ]:
# Ensemble Decoding
mean_svm_scores, splits_v_repeat_df, all_confusion_mats, (true_labels, pred_labels) = classifiers.ensemble_decoding(z_scored_combined_data, ensemble_averaging=False, n_repeats=5, num_splits=10)

In [ ]:
# Sliding Window Decoding
WINDOW = 2
mean_svm_scores, splits_v_repeat_df, all_confusion_mats, (true_labels, pred_labels) = classifiers.sliding_window_ensemble_decoding(z_scored_combined_data, window_size=WINDOW)

In [ ]:
output_dir = project_folder.analysis_dir.output_dir
if WINDOW:
    output_dir = output_dir.subdir(f'Window-{WINDOW}')

mean_scores_path = output_dir.joinpath('mean_svm_scores.pickle')
pd.to_pickle(mean_svm_scores, mean_scores_path)
splits_path = output_dir.joinpath('splits_v_repeat_df.pickle')
pd.to_pickle(splits_v_repeat_df, splits_path)
all_confusion_mat_path = output_dir.joinpath('all_confusion_mat.pickle')
pd.to_pickle(all_confusion_mats, all_confusion_mat_path)
labels_path = output_dir.joinpath('labels.pickle')
pd.to_pickle((true_labels, pred_labels), labels_path)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

mean_svm_scores = pd.read_pickle(r'./Analysis/Output/VGLUT/Window-5/mean_svm_scores.pickle')


In [ ]:
fig, ax = plt.subplots()
x_vals = np.round(np.linspace(-2, 7, len(mean_svm_scores)), 2)
x_labels = np.arange(-2, 7)
ax.plot(x_vals,mean_svm_scores)
ax.title.set_text('SVM Score')
ax.set_xlabel('Time (s)')
ax.set_ylabel('SVM Performance')
y_min = min(mean_svm_scores)*.95
y_max = max(mean_svm_scores)*1.05
ax.set_xlim([-2.5, 7.5])
ax.set_ylim([y_min, y_max])
# _ = ax.set_xticks(x_vals, rotation=45)
ax.vlines(x=[0], ymin=y_min, ymax=y_max, color=(1,0,0)) # FV On
ax.vlines(x=[2], ymin=y_min, ymax=y_max, color=(1,0,0)) # FV Off
fig.savefig(output_dir.joinpath('figure.pdf'), dpi=600)